[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JialinMao/private_CNN/blob/master/cifar10.ipynb)

In [1]:
!pip install opacus==1.1.1

In [3]:
!git clone https://github.com/JialinMao/private_CNN.git

Cloning into 'private_CNN'...
remote: Enumerating objects: 1625, done.
remote: Counting objects: 100% (1625/1625), done.
remote: Compressing objects: 100% (776/776), done.
remote: Total 1625 (delta 834), reused 1622 (delta 831), pack-reused 0
Receiving objects: 100% (1625/1625), 28.55 MiB | 22.42 MiB/s, done.
Resolving deltas: 100% (834/834), done.


In [6]:
cd private_CNN/

/content/private_CNN


In [24]:
!git clone https://github.com/kuangliu/pytorch-cifar.git

fatal: destination path 'pytorch-cifar' already exists and is not an empty directory.


In [ ]:
mv pytorch-cifar/models .


In [11]:
import sys
sys.path.insert(0, '/content/private_CNN')
import models
import private_CNN

In [10]:
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

from opacus.validators import ModuleValidator

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'


## Arguments

In [14]:
lr = 0.1
epochs = 20
bs = 128
eps = 5
grad_norm = 0.1
mode = 'ghost-mixed'
model = 'ResNet18'

## Loading Data

In [15]:
print('==> Preparing data..')

transform_train = transforms.Compose([
    transforms.ToTensor(),
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
])

trainset = torchvision.datasets.CIFAR10(
    root='../../data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=bs, shuffle=True, num_workers=2, drop_last=True)

testset = torchvision.datasets.CIFAR10(
    root='../../data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=bs, shuffle=False, num_workers=2)


==> Preparing data..


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ../../data/cifar-10-python.tar.gz to ../../data
Files already downloaded and verified


## Building Model

In [16]:
print('==> Building model..')
if "VGG" in model:
    net = models.VGG(model)
else:
    net = getattr(models, model)()

net = ModuleValidator.fix(net)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

print('number of parameters: ', sum([p.numel() for p in net.parameters()]))


==> Building model..
number of parameters:  11173962


## Privacy Engine

In [17]:
if 'ghost' in mode:
    criterion = nn.CrossEntropyLoss(reduction="none")
else:
    criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr,
                        momentum=0.9, weight_decay=5e-4)

if 'ghost' in mode:
    privacy_engine = private_CNN.PrivacyEngine(
        net,
        batch_size=bs,
        sample_size=len(trainloader.dataset),
        epochs=epochs,
        max_grad_norm=grad_norm,
        target_epsilon=eps,
        ghost_clipping=True,
        mixed='mixed' in mode,
    )
    privacy_engine.attach(optimizer)

## Trainining and Testing

In [18]:
# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(tqdm(trainloader)):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        if 'ghost' in mode:
            optimizer.step(loss=loss)
            loss = loss.mean()
        else:
            loss.backward()
            optimizer.step()

        train_loss += loss.mean().item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()


def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(tqdm(testloader)):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            if 'mixed' in mode:
                loss = loss.mean()
            test_loss += loss.mean().item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

In [ ]:
for epoch in range(epochs):
    train(epoch)
    test(epoch)